In [1]:
!pip install apache-beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 92.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.3/671.3 kB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 66.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 26.1 M

In [3]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.transforms.window import FixedWindows
from apache_beam.transforms.trigger import AfterCount, AfterProcessingTime
from apache_beam.transforms.window import TimestampedValue
from datetime import datetime

In [5]:
from apache_beam.options.pipeline_options import StandardOptions

options = PipelineOptions()
options.view_as(StandardOptions).streaming = True


In [13]:
def assign_random_timestamp(element):
    random_seconds_offset = random.randint(-3600, 3600)  # Offset within the last hour
    random_timestamp = (datetime.now() - datetime(1970, 1, 1)).total_seconds() + random_seconds_offset
    return beam.window.TimestampedValue(element, random_timestamp)


In [16]:
# Set up streaming options
options = PipelineOptions(flags=['--allow_unsafe_triggers'])
options.view_as(StandardOptions).streaming = True


# Define the Beam pipeline
p = beam.Pipeline(options=options)

(p | 'ReadFromCSV' >> beam.io.ReadFromText('/content/users_dataset.csv', skip_header_lines=1)
   | 'ParseCSV' >> beam.Map(lambda line: {'userid': line.split(',')[0], 'name': line.split(',')[1], 'age': line.split(',')[2]})
   | 'AssignRandomTimestamps' >> beam.Map(assign_random_timestamp)
   | 'WindowInto' >> beam.WindowInto(FixedWindows(3600), trigger=AfterCount(10), accumulation_mode=beam.transforms.trigger.AccumulationMode.DISCARDING)
   | 'GroupByAge' >> beam.Map(lambda x: (x['age'], 1))
   | 'CountByAge' >> beam.CombinePerKey(sum)
   | 'WriteResults' >> beam.io.WriteToText('output_results')
)

# Run the pipeline
result = p.run()
result.wait_until_finish()


'DONE'